In [7]:
import Pkg;
using SpecialFunctions
using Plots; plotly();
using Ipopt
using JuMP

In [8]:
#Initialize Variables

# m^3/g 
k_gac = 2
# g/m^3 
p_gac = 2.1*10^6 
p_soil = 1.3*10^6 
# m^3 
vol = 2*1.82*1.22 
#m_soil = p_soil*vol*(1-f_gac)
#m_gac = p_gac*vol*f_gac
lim = (600*10^-9)*(1.3*10^6)
Co = 2.8785

#m
x = 2

#m/s
v = 0.00000317
#m/hr
v_h = 0.011412

#m^2/s
D = 0.0001268392

#m^2/hr
D_h = 0.4566

unpolluted = (200*10^-9)*(1.3*10^6) 
weaklypolluted = (600*10^-9)*(1.3*10^6)
heavilypolluted = (1000*10^-9)*(1.3*10^6)

1.3000000000000005

In [19]:
#No GAC Function
function conc(Co,x,v,t,D)
    C = (Co)*erfc((x - v*t)/(2*sqrt(D*t)))
    return C
end

conc (generic function with 1 method)

In [30]:
#No GAC Concentration Curve
figure1 = plot();
        plot!(figure1, [conc(2.8785,2,0.011412,t,0.4566) for t ∈ 0:8], xlabel = "time [hr]",ylabel = "Concentration [g/m^3]", ylim = [0,1.5])
plot!(figure1, fg_legend = :transparent, legend = :bottomright,ylim= [0,1.5])
hline!([unpolluted], label = "unpolluted")
hline!([weaklypolluted], label = "weakly polluted")
hline!([heavilypolluted], label = "heavily polluted")

<!DOCTYPE html>
 
 
 Plots.jl

In [10]:
#GAC Function
function gac(Co,x,v,t,D,f_gac)
        k_d = k_gac*f_gac
        p_d = 1.3
        theta = 0.5
        R = 1 + (k_d*p_d)/theta 
        C = (Co)*erfc((x -(v/R)*t)/(2*sqrt((D/R)*t)))
    return C
end

gac (generic function with 1 method)

In [49]:
#GAC Concentration Curves
figure = plot(); 
for f_gac=[.05,.1,.2,.3,.4,.5]
		plot!(figure, [gac(2.8785,2,0.011412,t,0.4566,f_gac) for t ∈ 0:16], xlabel = "time [hr]",ylabel = "Concentration [g/m^3]", ylim = [0,1.5])
	end

plot!(figure, fg_legend = :transparent, legend = :bottomright,ylim= [0,1.5], legendtitle ="f_gac")
hline!([unpolluted], label = "unpolluted")
hline!([weaklypolluted], label = "weakly polluted")
hline!([heavilypolluted], label = "heavily polluted")

<!DOCTYPE html>
 
 
 Plots.jl

In [51]:
C_1 = gac(2.8785,2,0.011412,10,0.4566,0.05)

1.3763918274513416

In [40]:
# Determine the number of times the GAC needs to be replaced in a year 
function time(f_gac)
    t = 9.5
    C = gac(2.8785,2,0.011412,t,0.4566,f_gac)
    if C < (1000*10^-9)*(1.3*10^6)
        t = t+0.1
        C = gac(2.8785,2,0.011412,t,0.4566,f_gac)
    end
    return t
end
#t = 5401 
#opt_frac = 0.1
#count = 8760/t
# Cost of GAC [$/ft^3]
#price = 99/0.02831685

time (generic function with 1 method)

In [41]:
time(0.1)

9.6

In [14]:
cost = (price*count*vol*opt_frac)

2518.1488469653023